In [5]:
using LinearAlgebra

In [1]:
# Cosine Similarity for determining how "similar" two vectors are. 
# Why use cosine similarity? Because in with eigenvectors, magnitude does not matter, only direction. 

# inputs: a, b are two vectors of the same dimension
# \result: [-1, 1]
function cosine_similarity(a::Vector, b::Vector)
    return dot(a, b) / (norm(a) * norm(b))
end


cosine_similarity (generic function with 1 method)

In [2]:
# Power method for finding the largest eigenvector/value pair

# A:            matrix we are going to calculate with
# u_0:          initial vector
# tolerance:    necessary level of accuracy before returning, value in [0, 1]
# oracle_vec:   oracle eigenvector for what we are calculating
# \result:      eigenvetor, eigenvalue, number of iterations to reach tolerance
function power_method(A, u_k, oracle_vector, tolerance, with_print=false) 

    println("Starting Power Method")
    # 1. Calculate similarity
    similarity = cosine_similarity(u_k, oracle_vector)
    num_iterations = 0 

    # 2. While similarity is not of certain point...
    while (similarity < 1 - tolerance)
        
        # 2.a let u_k' = A u_k
        u_k = A * u_k

        # normalize u_k, this is done to prevent u_k from getting too large or too small, avoiding instablities
        u_k /= norm(u_k)

        # 2.b update similarity
        similarity = cosine_similarity(u_k, oracle_vector)

        # 2.c increment k
        num_iterations += 1

        if with_print
            println("Iteration: ", num_iterations, " Similarity: ", similarity) 
        end
    end


    # 3. We know vectors are similar enough, so return calculated vector, eigenvector, and num iterations

    # 3.a calculate eigen value
    # This has been done using Rayleigh Quotient (https://en.wikipedia.org/wiki/Rayleigh_quotient)
    eigenvalue = dot(u_k, A * u_k) / dot(u_k, u_k)

    return u_k, eigenvalue, num_iterations
end

power_method (generic function with 2 methods)

In [ ]:
# This is a naieve implementation of calculating the smallest eigenvector/eigenvalue pair. 
# This is a poor implementation because it involves calculating the inverse of A - mu*I
# An enhanced implementation would store the LU decompositions to do this

function naive_inverse_power_method(A, u_k, mu, oracle_vector, tolerance, with_print=false)
    println("Starting Inverse Power Method")

    # Adjust matrix A by the shift mu
    n = size(A, 1)
    A_shifted = A - mu * I

    # 1. Calculate similarity
    similarity = cosine_similarity(u_k, oracle_vector)
    num_iterations = 0 

    # 2. While similarity is not of certain point...
    while (similarity < 1 - tolerance)
        
        # 2.a solve (A - mu*I)u_k' = u_k
        # Here we use a linear solver instead of direct multiplication
        u_k = A_shifted \ u_k

        # Normalize u_k
        u_k /= norm(u_k)

        # 2.b update similarity
        similarity = cosine_similarity(u_k, oracle_vector)

        # 2.c increment k
        num_iterations += 1

        if with_print
            println("Iteration: ", num_iterations, " Similarity: ", similarity) 
        end
    end

    # 3. Calculate eigenvalue using Rayleigh Quotient
    eigenvalue = dot(u_k, A * u_k) / dot(u_k, u_k)

    return u_k, eigenvalue, num_iterations
end


In [3]:
function inverse_power_method_LU(A, u_k, oracle_vector, tolerance, with_print=false)
    println("Starting Inverse Power Method with LU Decomposition")

    # set mu
    mu = dot(u_k, A * u_k) / dot(u_k, u_k) # Rayleigh Quotient

    # Shift matrix A by mu
    n = size(A, 1)
    A_shifted = A - mu * I

    # Perform LU decomposition of A_shifted
    # Julia's factorize function automatically chooses the best factorization
    LU = factorize(A_shifted)

    # Initial similarity calculation
    similarity = cosine_similarity(u_k, oracle_vector)
    num_iterations = 0 

    while (similarity < 1 - tolerance)
        # Solve the system using LU factors
        u_k = LU \ u_k

        # Normalize u_k
        u_k /= norm(u_k)

        # Update similarity
        similarity = cosine_similarity(u_k, oracle_vector)

        # Increment iteration count
        num_iterations += 1

        if with_print
            println("Iteration: ", num_iterations, " Similarity: ", similarity) 
        end
    end

    # Calculate eigenvalue using Rayleigh Quotient
    eigenvalue = dot(u_k, A * u_k) / dot(u_k, u_k)

    return u_k, eigenvalue, num_iterations
end


inverse_power_method_LU (generic function with 2 methods)

In [7]:
A = diagm(0 => [4, 3, 2])
u_0 = rand(3)
largest_oracle_vector = [1, 0, 0] # Since the largest eigenvalue is 4, corresponding eigenvector is [1, 0, 0]
smallest_oracle_vector = [0, 0, 1] # Since the largest eigenvalue is 4, corresponding eigenvector is [1, 0, 0]
tolerance = 1e-3

apx_eigenvec, apx_eigenval, num_iterations = power_method(A, u_0, largest_oracle_vector, tolerance)
println("<-- Power Methods results for Diagonal Matrix -->")
println("Oracle Eigenvector: ", largest_oracle_vector)
println("Oracle Eigenvalue: ", dot(largest_oracle_vector, A * largest_oracle_vector) / dot(largest_oracle_vector, largest_oracle_vector))
println("Approximated Eigenvector: ", apx_eigenvec)
println("Approximated Eigenvalue: ", apx_eigenval)
println("Number of Iterations: ", num_iterations)

println("")

apx_eigenvec, apx_eigenval, num_iterations = inverse_power_method_LU(A, u_0, smallest_oracle_vector, tolerance)
println("<-- Inverse Power Methods results for Diagonal Matrix -->")
println("Oracle Eigenvector: ", smallest_oracle_vector)
println("Oracle Eigenvalue: ", dot(smallest_oracle_vector, A * smallest_oracle_vector) / dot(smallest_oracle_vector, smallest_oracle_vector))
println("Approximated Eigenvector: ", apx_eigenvec)
println("Approximated Eigenvalue: ", apx_eigenval)
println("Number of Iterations: ", num_iterations)


In [ ]:
A = [2 1; 1 2]
u_0 = rand(2)
oracle_vector = [1/sqrt(2), 1/sqrt(2)] # Assuming knowledge of the eigenvector for testing
tolerance = 1e-6

apx_eigenvec, apx_eigenval, num_iterations = power_method(A, u_0, oracle_vector, tolerance)

println("<-- Result for Diagonal Matrix -->")
println("Oracle Eigenvector: ", oracle_vector)
println("Oracle Eigenvalue: ", dot(oracle_vector, A * oracle_vector) / dot(oracle_vector, oracle_vector))
println("Approximated Eigenvector: ", apx_eigenvec)
println("Approximated Eigenvalue: ", apx_eigenval)
println("Number of Iterations: ", num_iterations)
